In [ ]:
from itertools import chain
p = list(route_df.route)
p_set = set(list(chain.from_iterable(p)))
p_df = pd.DataFrame({'id' : list(p_set)})
p_df['x'] = p_df.id.apply(lambda x: GG.nodes[x]['x'])
p_df['y'] = p_df.id.apply(lambda x: GG.nodes[x]['y'])
p_df

In [ ]:
routes = ox.k_shortest_paths(GG, 278364190, 291750883, k=30, weight="length")
list(routes)

route = nx.shortest_path(GG, 278364190, 291750883)
route
fig, ax = ox.plot_graph_routes(GG, list(df_1[['nodes']].values[0]), route_colors='r', route_linewidth=6, node_size=0)
fig, ax = ox.plot_graph_route(full, list(df_1[['nodes']].values[0])[0], route_linewidth=6, node_size=0, bgcolor='k')



In [ ]:
GG = ox.graph_from_place('Tallin', retain_all=True, simplify = True, network_type='drive')

df_1 = full_feature_df[full_feature_df['kind_of'] == 'building'].reset_index(drop=True)
df_1 = df_1[df_1['place'] == 'Tallin'].reset_index(drop=True)
one_route = []
list_f = []
list_t = []
for i in range(df_1.shape[0]):
    for j in range(i + 1, df_1.shape[0]):
        one_route.append(nx.shortest_path(full, df_1['new_nodes'].iloc[i], df_1['new_nodes'].iloc[j]))
        list_f.append(df_1['name'].iloc[i])
        list_t.append(df_1['name'].iloc[j])

route_df = pd.DataFrame({'from' : list_f, 'to' : list_t, 'route' : one_route})
route_df.sample(2)



routes = ox.k_shortest_paths(GG, 278364190, 291750883, k=30, weight="length")
# fig, ax = ox.plot_graph_routes(
#     GG, list(routes), route_colors="y",
#     route_linewidth=4, node_size=0)
df_1.head(2)

**Построение графа с узлами в нужных точках**

In [ ]:
list_graphs = []

for i in range(route_df.route.shape[0]):
    
    r = route_df['route'].values[i]

    rr = pd.DataFrame({'new_nodes' : r})
    rr['osmid'] = rr['new_nodes']
    rr = rr.set_index('osmid')
    rr['x'] = rr.new_nodes.apply(lambda x: GG.nodes[x]['x'])
    rr['y'] = rr.new_nodes.apply(lambda x: GG.nodes[x]['y'])
    rr

    edge_dict = {'u': rr.new_nodes[:-1].values, 'v': rr.new_nodes[1:].values, 'key': 0}
    edge_gdf = gd.GeoDataFrame(edge_dict, crs=None)
    edge_gdf = edge_gdf.set_index(['u', 'v', 'key'])

    graph_attrs = {"crs": "WGS84"}
    multi_digraph_aero = ox.convert.graph_from_gdfs(
        rr, edge_gdf, graph_attrs=graph_attrs)
    
    list_graphs.append(multi_digraph_aero)
    
my_graph = list_graphs[0]
for i in range(1, len(list_graphs)):
    my_graph = nx.compose_all([my_graph, list_graphs[i]])

ox.plot_graph(my_graph)

In [ ]:
geo_dataframe = ox.graph_to_gdfs(
    my_graph, nodes=True, edges=True,
    node_geometry=False, fill_edge_geometry=False)

nodes, edges = ox.graph_to_gdfs(my_graph)

list_nodes = list(df_1['new_nodes'].values)

n = nodes[nodes['new_nodes'].isin(list_nodes)]
g = ox.convert.graph_from_gdfs(n, edges, graph_attrs=None)
ox.plot_graph(g)

In [ ]:
''' Перепись функции тегов на цикл '''

name_city = 'Tallin'
gdf = ox.features_from_place(name_city, tags_build).reset_index()
gdf = gdf[['element_type', 'osmid']]
gdf['n_osmid'] = gdf['element_type'].apply(lambda x: x[0]) + gdf['osmid'].astype(str)

list_lat_lon = []
try:
    for i in range(gdf.n_osmid.shape[0]):
        bb = ox.geocode_to_gdf(gdf.n_osmid.iloc[i], by_osmid=True)[['lat', 'lon']]
        list_lat_lon.append(bb.iloc[:])
    dfs = pd.concat(list_lat_lon, axis=0).reset_index(drop=True)
    gdf = gdf.merge(dfs, on=dfs.index).drop('key_0', axis=1)
    gdf = gdf[['lat', 'lon']]
    gdf['kind_of'] = list(tags_build.keys())[0]
except:
    pass


name_city = 'Tallin'
gdf = ox.features_from_place(name_city, tags_build).reset_index()
gdf = gdf[['element_type', 'osmid']]
gdf['n_osmid'] = gdf['element_type'].apply(lambda x: x[0]) + gdf['osmid'].astype(str)

list_lat_lon = []
try:
    for i in range(gdf.n_osmid.shape[0]):
        bb = ox.geocode_to_gdf(gdf.n_osmid.iloc[i], by_osmid=True)[['lat', 'lon']]
        list_lat_lon.append(bb.iloc[:])
    dfs = pd.concat(list_lat_lon, axis=0).reset_index(drop=True)
    gdf = gdf.merge(dfs, on=dfs.index).drop('key_0', axis=1)
    gdf = gdf[['lat', 'lon']]
    gdf['kind_of'] = list(tags_build.keys())[0]
except:
    pass
gdf

In [ ]:
    # query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s)" % (table, cols)

    # query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, (ST_GeomFromEWKT(SRID=4269;%%s)))" % (table, cols)
    # query = """INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, ST_GeomFromWKB(%%s::geometry, 4326))""" % (table, cols)
    # insert_query = """INSERT INTO %s(%s) VALUES (ST_GeomFromWKB(%(geom)s::geometry, 4326), %(total_time)s, %(id)s)"""

    query = "INSERT INTO example_table_nodes (osmid, x, y, geometry) VALUES(%s,%s,%s, (ST_GeomFromEWKT(SRID=4269;%s)))" % (tuples[0], tuples[1], tuples[2], tuples[3])
    query = "INSERT INTO example_table_nodes (osmid, x, y, geometry) VALUES(%s,%s,%s, (ST_GeomFromEWKT(SRID=4269;%s)))"
    # print("INSERT INTO %s(%s) VALUES(%s,%s,%s, (ST_GeomFromEWKT(SRID=4269;%s)))" % (table, cols, tuples))

In [ ]:
    # arr_x = df.to_numpy()[:, 1]
    # arr_y = df.to_numpy()[:, 2]
    # arr_id = df.to_numpy()[:, 0]
    # arr_g = df.to_numpy()[:, 3]
    # query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, (ST_GeomFromEWKT(SRID=4269;%%s)))" % (table, cols)
    # query = """INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, ST_GeomFromWKB(%%s::geometry, 4326))""" % (table, cols)
    # insert_query = """INSERT INTO %s(%s) VALUES (ST_GeomFromWKB(%(geom)s::geometry, 4326), %(total_time)s, %(id)s)"""

    # query = "INSERT INTO example_table_nodes (osmid, x, y, geometry) VALUES(%s,%s,%s, (ST_GeomFromEWKT(SRID=4269;%s)))" % (tuples[0], tuples[1], tuples[2], tuples[3])
    # query = "INSERT INTO example_table_nodes (osmid, x, y, geometry) VALUES({},{},{}, (ST_SetSRID(ST_POINT({}, {}), 4326)))".format(arr_id[0], arr_x[0], arr_y[0], arr_x[0], arr_y[0])
    
    # query = "INSERT INTO example_table_nodes (osmid, x, y, geometry) VALUES({},{},{},ST_GeomFromText('POINT(-71.060316 48.432044), 4326)')".format(arr_id[0], arr_x[0], arr_y[0])
    # query = """INSERT INTO example_table_nodes(osmid, x, y, geometry) VALUES({},{},{},ST_SetSRID(ST_MakePoint(-71.060316 48.432044), 4326))""".format(arr_id[0], arr_x[0], arr_y[0])
    
    # query = """INSERT INTO example_table_nodes(geom) VALUES (ST_AsText('POINT (24.7071351 59.4619055)'))"""
    # print("INSERT INTO %s(%s) VALUES(%s,%s,%s, (ST_GeomFromEWKT(SRID=4269;%s)))" % (table, cols, tuples))


In [ ]:
from shapely.wkt import loads
# df_nodes_riga['geometry'] = df_nodes_riga['geometry'].apply(lambda x: loads(x))
# df_edges_riga['geometry'] = df_edges_riga['geometry'].apply(lambda x: loads(x))

In [ ]:
import osmnx as ox
import pandas as pd
from shapely import wkt
import geopandas as gd

gdf_nodes = pd.read_csv('data/nodes_tallin.csv')
gdf_edges = pd.read_csv('data/edges_tallin.csv')

gdf_nodes_new = gdf_nodes.set_index('osmid').drop(['new_nodes'], axis=1)
gdf_nodes_new = gdf_nodes_new[['y', 'x', 'geometry']]
gdf_edges_new = gdf_edges.set_index(['u', 'v', 'key'])

gdf_edges_new['geometry'] = gdf_edges_new['geometry'].apply(wkt.loads)
gdf_nodes_new['geometry'] = gdf_nodes_new['geometry'].apply(wkt.loads)

df_edges = gd.GeoDataFrame(gdf_edges_new, crs = 'WGS84')
df_nodes = gd.GeoDataFrame(gdf_nodes_new, crs = 'WGS84')

G = ox.graph_from_gdfs(df_nodes, df_edges)

ox.plot_graph(G)